In [6]:
from validator.InstanceCO22 import InstanceCO22
import warnings
from collections import OrderedDict
warnings.filterwarnings( "ignore", module = "matplotlib\..*" )
from util import *
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from vrpy import VehicleRoutingProblem
import pprint
import warnings
import logging
import llist
import pandas as pd
from typing import List, Set, Tuple, Dict
import random
import time
import copy
import initialSolutions
from dataClasses import * 
from cachetools import cached
import initialSolutions
warnings.filterwarnings("ignore", module="vrpy\..*")


logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.CRITICAL)
%reload_ext autoreload
%autoreload 2

In [7]:
instance = loadInstance(10)

In [11]:
#test DayHubRoutes distance and demand calculation 

#add all requests in one route orderer by reqID
nodes = [RequestNode(_.ID, _.customerLocID, _.amounts, instance.Locations[_.customerLocID-1].X, instance.Locations[_.customerLocID-1].Y, isHub=False) for _ in instance.Requests]
dm = DistanceMatrix(instance)

route1 = HubRoute(routeID=1, hubLocID=1)
route1.addNodes(nodes[0:10])

route2 = HubRoute(routeID=2, hubLocID=2)
route2.addNodes(nodes[10:20])

dayHubRoutes = DayHubRoutes()
dayHubRoutes.addRoute(route1)
dayHubRoutes.addRoute(route2)
dayHubRoutes.addRoute(route2)
print(dayHubRoutes.hubsUsed())
print(dayHubRoutes.nHubs())
print(dayHubRoutes.isFeasible(instance.VanCapacity, instance.VanMaxDistance, dm, instance, verbose=True))
print(dayHubRoutes.coversRequestIDs([1,2,3,4,5]))


#random merge
route1 = HubRoute(routeID=1, hubLocID=1)
route1.addNodes(nodes[0:3])

route2 = HubRoute(routeID=2, hubLocID=2)
route2.addNodes(nodes[3:6])

dayHubRoutes = DayHubRoutes()
dayHubRoutes.addRoute(route1)
dayHubRoutes.addRoute(route2)

print("-"*20)
print(dayHubRoutes)
print("-"*10)
dayHubRoutes.randomMergeRoutes()
print(dayHubRoutes)

#random section insertion
route1 = HubRoute(routeID=1, hubLocID=1)
route1.addNodes(nodes[0:5])

route2 = HubRoute(routeID=2, hubLocID=2)
route2.addNodes(nodes[5:10])

dayHubRoutes = DayHubRoutes()
dayHubRoutes.addRoute(route1)
dayHubRoutes.addRoute(route2)

print("-"*20)
print(dayHubRoutes)
print("-"*10)
dayHubRoutes.randomSectionInsertion()
print(dayHubRoutes)


{1, 2}
2
True
True
--------------------
HubRoute from hub 1: Node(reqID:1, locID:14) -> Node(reqID:2, locID:21) -> Node(reqID:3, locID:14)
HubRoute from hub 2: Node(reqID:4, locID:27) -> Node(reqID:5, locID:23) -> Node(reqID:6, locID:13)

----------
HubRoute from hub 2: Node(reqID:4, locID:27) -> Node(reqID:5, locID:23) -> Node(reqID:6, locID:13) -> Node(reqID:1, locID:14) -> Node(reqID:2, locID:21) -> Node(reqID:3, locID:14)

--------------------
HubRoute from hub 1: Node(reqID:1, locID:14) -> Node(reqID:2, locID:21) -> Node(reqID:3, locID:14) -> Node(reqID:4, locID:27) -> Node(reqID:5, locID:23)
HubRoute from hub 2: Node(reqID:6, locID:13) -> Node(reqID:7, locID:24) -> Node(reqID:8, locID:24) -> Node(reqID:9, locID:16) -> Node(reqID:10, locID:14)

----------
HubRoute from hub 1: Node(reqID:1, locID:14) -> Node(reqID:5, locID:23) -> Node(reqID:2, locID:21) -> Node(reqID:3, locID:14) -> Node(reqID:4, locID:27)
HubRoute from hub 2: Node(reqID:6, locID:13) -> Node(reqID:7, locID:24) -> N

In [8]:

nodes = [HubNode(loc.ID, [1, 1, 1], loc.X, loc.Y) for loc in instance.Locations]
dm = DistanceMatrix(instance)

route1 = DepotRoute(routeID=1)
route1.addNodes(nodes[2:3])

assert route1.length(dm) == 124.0

route2 = DepotRoute(routeID=1)
route2.addNodes(nodes[0:10])


assert route2.demand() == 30

dayDepotRoutes = DayDepotRoutes()
dayDepotRoutes.addRoute(route1)
dayDepotRoutes.addRoute(route2)


In [9]:
print(instance.Locations[0].X)
print(instance.Locations[0].Y)
print(instance.Locations[2:3][0].X)
print(instance.Locations[2:3][0].Y)

30
53
89
69


In [10]:
hubRoutes = initialSolutions.solveHub(instance)

In [11]:
depotRoutes = initialSolutions.solveDepotDC(instance, hubRoutes)

In [12]:
depotRoutes.computeCost(truckDistanceCost=instance.TruckDistanceCost, truckDayCost=instance.TruckDayCost, truckCost=instance.TruckCost, distanceMatrix=dm)

52760.0

In [13]:
depotRoutes.isFeasible(truckCapacity=instance.TruckCapacity, truckMaxDistance=instance.TruckMaxDistance, dm=dm, instance=instance)

True

In [17]:
solution = Solution(hubRoutes, depotRoutes)

In [18]:

solution.computeCost(instance=instance,hubCost=True, depotCost=True, distanceMatrix=dm, useHubOpeningCost=True)

148230.0

In [59]:
solution.computeDepotSolution(instance)

In [60]:
solution.computeCost(instance=instance,hubCost=True, depotCost=True, distanceMatrix=dm, useHubOpeningCost=True)

148230.0

In [79]:
solution.isFeasible(instance=instance, distanceMatrix=dm, useCheckHubCanServe=True)

True

In [21]:
solution.toStr(instance)

'DATASET = CO2022_11 \n \nDAY = 1 \nNUMBER_OF_TRUCKS = 2 \n0 H7 0,9,16 H6 12,10,7 \n1 H1 3,14,5 H4 8,4,8 H9 3,15,5 H10 0,2,0 H8 7,0,2 \nNUMBER_OF_VANS = 2 \n1 H7 2 \n2 H6 1 \n\nDAY = 2 \nNUMBER_OF_TRUCKS = 0 \nNUMBER_OF_VANS = 5 \n1 H4 7 8 \n2 H6 3 9 \n3 H7 5 \n4 H9 4 \n5 H10 6 \n\nDAY = 3 \nNUMBER_OF_TRUCKS = 0 \nNUMBER_OF_VANS = 4 \n1 H8 14 \n2 H1 12 16 13 \n3 H9 11 \n4 H6 10 15 \n\nDAY = 4 \nNUMBER_OF_TRUCKS = 0 \nNUMBER_OF_VANS = 2 \n1 H7 17 \n2 H9 18 19 \n\nDAY = 5 \nNUMBER_OF_TRUCKS = 2 \n0 H8 8,0,3 H7 0,10,16 H6 7,0,9 \n1 H10 21,12,10 H9 2,12,16 H4 9,20,7 H1 0,3,5 \nNUMBER_OF_VANS = 3 \n1 H10 21 \n2 H1 20 \n3 H9 22 \n\nDAY = 6 \nNUMBER_OF_TRUCKS = 0 \nNUMBER_OF_VANS = 4 \n1 H7 24 \n2 H8 26 \n3 H10 25 \n4 H4 23 \n\nDAY = 7 \nNUMBER_OF_TRUCKS = 0 \nNUMBER_OF_VANS = 4 \n1 H9 31 32 \n2 H10 28 \n3 H4 29 30 \n4 H6 27 \n\nDAY = 8 \nNUMBER_OF_TRUCKS = 0 \nNUMBER_OF_VANS = 4 \n1 H7 33 38 \n2 H8 37 \n3 H10 34 35 \n4 H4 36 \n\nDAY = 9 \nNUMBER_OF_TRUCKS = 2 \n0 H1 0,10,6 H4 7,0,4 H9 11,22,

In [95]:
%%time
solution.computeDepotSolution(instance)

CPU times: user 11 µs, sys: 1e+03 ns, total: 12 µs
Wall time: 14.3 µs


In [64]:
#caching
recomputed = initialSolutions.solveDepotDCCached(instance,hubRoutes,cache=depotRoutes, redo=[1])

1


In [67]:
solution.depotRecomputeNecessary[1] = True
solution.computeDepotSolution(instance)

1


In [72]:
solution.hubIntraDayRandomMergeRoutes(day=1)
solution.recomputeNecessary()

True

In [96]:
for i in range(20):
    s = time.time()
    solution.hubRandomMoveNodeDayEarly()
    solution.computeDepotSolution(instance)
    print(time.time()-s)

17
0.533607006072998
13
0.16094398498535156
13
0.1794288158416748
17
0.22362804412841797
13
0.1751251220703125
9
0.404674768447876
9
0.3573648929595947
9
0.24307680130004883
5
0.6459648609161377
13
0.17805218696594238
17
0.22802996635437012
9
0.2350757122039795
5
0.7847230434417725
17
0.21874094009399414
9
0.2156810760498047
13
0.1862640380859375
5
0.6998581886291504
9
0.30921387672424316
1
0.39078783988952637
9
0.2061760425567627
